In [2]:
# Tensorly library
import tensorly as tl
import numpy as np
from functools import reduce

In [28]:
"""
TENSOR TO TENSOR REGRESSION v1    
  Inputs:
    X - Input Tensor, shape (N, I_1, ..., I_{M1})
    Y - Output Tensor, shape (N, J_1, ..., J_{M2})
    rank - The rank of the solution weights matrix
    lambda_reg - how much l2 regularization to use
    return_factors - whether to include the weight matrix factors along with the weight matrix itself
    eps - the precision of our estimate
    
  Outputs:
    W - weight matrix tensor, shape (I_1, ..., I_{M1}, J_1, ..., J_{M2})
    factors - List of CP factors which forms the CP decomposition of W. Will only be returned if return_factors==True
"""
def tensor_to_tensor_regression_1(X, Y, rank, lambda_reg=0.0, return_factors=False, eps=1e-4):
    # Check that the N is consistent between X and Y tensors
    if X.shape[0] != Y.shape[0]:
        print("Wrong leading dimensions for tensors X and Y")
    
    # Number of examples (leading dimension in the tensor)
    N = X.shape[0]
    
    # Setup the sizes of the X and Y matrices
    I = reduce(lambda x, y: x * y, X.shape[1:])
    J = reduce(lambda x, y: x * y, Y.shape[1:])
    
    # Initialize the u_r and v_r vectors which we will directly optimize
    u_rs = [np.random.random(I)-0.5 for r in range(rank)]
    v_rs = [np.random.random(J)-0.5 for r in range(rank)]
    
    # Matricize X and Y
    X_mat = tl.unfold(X, mode=0)
    Y_mat = tl.unfold(Y, mode=0)
    
    # Precompute some repeatedly used matrices
    XtXpL = np.matmul(np.transpose(X_mat), X_mat) + lambda_reg * np.identity(I)
    XtXpL_inv = np.linalg.inv(XtXpL)
    XtY = np.matmul(np.transpose(X_mat), Y_mat)
    YtX = np.transpose(XtY)
    
    steps = 0
    prev_error = -1
    errors = [] # TODO: Remove for production (only used for testing)
    # Keep optimizing until the error has converged
    while steps < 100 and abs(prev_error) >= 1:
        # For every rank of the weights matrix
        for r in range(rank):
            u_comp = np.zeros(I)
            v_comp = np.zeros(J)
            # Do some ugly linear algebra
            for r1 in range(rank):
                # Only want cases where r1 != r
                if r1 == r:
                    continue
                # Computing the summation in equations (13) and (14)
                u_comp += np.dot(v_rs[r1], v_rs[r]) * np.matmul(XtXpL, u_rs[r1])
                v_comp += np.matmul(u_rs[r], np.matmul(XtXpL, u_rs[r1])) * v_rs[r1]
            # More ugly linear algebra, finishing off the calculation from 13 and 14
            u_r_new = np.matmul(XtXpL_inv, np.matmul(XtY, v_rs[r])-(u_comp / 2)) / np.dot(v_rs[r], v_rs[r])
            v_r_new = (np.matmul(YtX, u_rs[r]) - (v_comp / 2)) / np.matmul(u_rs[r], np.matmul(XtXpL, u_rs[r]))
            # Actually update the u_rs/v_rs
            u_rs[r] = u_r_new
            v_rs[r] = v_r_new
        
        # Compute the new error, this time ignoring regularization
        W_mat = np.zeros((I, J)) # TODO: implement
        for r in range(rank):
            # Add each W_r
            W_mat += tl.tenalg.kronecker([u_rs[r], v_rs[r]]).reshape(I, J)
        error = np.square(Y_mat - np.matmul(X_mat, W_mat)).mean()
        errors.append(error)
        # Determine if we have converged
        if prev_error > 0 and abs(prev_error - error) < eps:
            print("Converged after", steps, "steps. Final Error:", error)
            break
        else:
            print("Step:", steps, "Error:", error)
        # Reset the previous error
        prev_error = error
        # Next step
        steps += 1
        
    # Now we have converged, so return the W matrix
    W = np.zeros((X.shape[1:]) + (Y.shape[1:])) # TODO: implement
    # TODO: Add functionality to make the factors
    return W_mat
    

In [36]:
N = 20
X = np.random.random((N, 5, 7))
Y = np.zeros((N, 2, 3))
# Setup Y tensor with some dummy data
for n in range(N):
    x = X[n]
    Y[n] = np.array([[x[0, 0] + x[1, 1] , 2 * x[1, 0] - x[3, 2], (x[4, 5] + 1) ** 2],
                    [-x[1, 6] + 3 * x[1, 5] ,  - x[0, 6] - x[3, 3], (x[2, 5] + 2) ** 2]])
# Now fit it
tensor_to_tensor_regression_1(X, Y, 10, lambda_reg=1000)

Step: 0 Error: 9.223671117250511
Step: 1 Error: 5.077744409767049
Step: 2 Error: 6088.795936309728
Step: 3 Error: nan


C:\Users\alexg\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: RuntimeWarning: overflow encountered in multiply
C:\Users\alexg\Anaconda3\lib\site-packages\ipykernel_launcher.py:57: RuntimeWarning: overflow encountered in multiply
C:\Users\alexg\Anaconda3\lib\site-packages\ipykernel_launcher.py:59: RuntimeWarning: invalid value encountered in matmul


array([[nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, na

In [39]:
arr1 = np.array([[1, 2, 3]])
arr2 = np.array([[5, 7, 4, 5]])
tl.tenalg.kronecker([arr1, arr2]).reshape((3, 4))

array([[ 5,  7,  4,  5],
       [10, 14,  8, 10],
       [15, 21, 12, 15]])